In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import ResNet50
import helper

from snn import SNNGenerator, SNN

In [ ]:
helper.download_celeba()

In [ ]:
from partitioning import build_celeba_partitions
build_celeba_partitions()

In [ ]:
from pairing import build_celeba_pairs
build_celeba_pairs()

In [ ]:
def load_images(paths):
    images = [helper.get_image(path, 80, 80, 'RGB').astype(np.float)/255 for path in paths]
    return np.array(images)

In [ ]:
class CelebAGenerator(SNNGenerator):
    def __init__(self, genuine_pairs_df: pd.DataFrame, impostor_pairs_df: pd.DataFrame, batch_size: int):
        self.__genuine_pairs_df__ = genuine_pairs_df
        self.__impostor_pairs_df__ = impostor_pairs_df
        self.__batch_size__ = batch_size
    
    def __len__(self):
        return (self.__genuine_pairs_df__.shape[0]+self.__impostor_pairs_df__.shape[0])//self.__batch_size__

    def __getitem__(self, index: int):
        def get_genuines():
            my_rows = self.__genuine_pairs_df__.sample(n=self.__batch_size__//2, replace=False)
            images_a = load_images([os.path.join('celeba', 'img_align_celeba', fn) for fn in list(my_rows['file_a'])])
            images_b = load_images([os.path.join('celeba', 'img_align_celeba', fn) for fn in list(my_rows['file_b'])])
            images_y = np.ones(shape=(self.__batch_size__//2,1))
            return images_a, images_b, images_y

        def get_impostors():
            my_rows = self.__impostor_pairs_df__.sample(n=self.__batch_size__//2, replace=False)
            images_a = load_images([os.path.join('celeba', 'img_align_celeba', fn) for fn in list(my_rows['file_a'])])
            images_b = load_images([os.path.join('celeba', 'img_align_celeba', fn) for fn in list(my_rows['file_b'])])
            images_y = np.zeros(shape=(self.__batch_size__//2,1))
            return images_a, images_b, images_y

        genuines_a, genuines_b, genuines_y = get_genuines()
        impostors_a, impostors_b, impostors_y = get_impostors()
        a = np.append(genuines_a, impostors_a, axis=0)
        b = np.append(genuines_b, impostors_b, axis=0)
        y = np.append(genuines_y, impostors_y, axis=0)
        return ([a, b], y)

In [ ]:
training_genuine_pairs = pd.read_csv(os.path.join('celeba_pairs', 'training_genuine_pairs.csv'))
training_impostor_pairs = pd.read_csv(os.path.join('celeba_pairs', 'training_impostor_pairs.csv'))
eval_genuine_pairs = pd.read_csv(os.path.join('celeba_pairs', 'eval_genuine_pairs.csv'))
eval_impostor_pairs = pd.read_csv(os.path.join('celeba_pairs', 'eval_impostor_pairs.csv'))

In [ ]:
divider = 100
training_genuine_pairs = training_genuine_pairs.sample(n=training_genuine_pairs.shape[0]//divider, random_state=1)
training_impostor_pairs = training_impostor_pairs.sample(n=training_impostor_pairs.shape[0]//divider, random_state=1)
eval_genuine_pairs = eval_genuine_pairs.sample(n=eval_genuine_pairs.shape[0]//divider, random_state=1)
eval_impostor_pairs = eval_impostor_pairs.sample(n=eval_impostor_pairs.shape[0]//divider, random_state=1)


In [ ]:
def build_resnet50_encoder(n_features=100):
    base_model = ResNet50(weights=None, include_top=False, input_shape=(80,80,3))
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.7)(x)
    x = layers.Dense(n_features, activation='linear')(x)
    model = keras.Model(inputs = base_model.input, outputs = x)
    return model

In [ ]:
my_snn = SNN((80,80,3),build_resnet50_encoder())

In [ ]:
training_generator = CelebAGenerator(training_genuine_pairs, training_impostor_pairs, 50)
eval_generator = CelebAGenerator(eval_genuine_pairs, eval_impostor_pairs, 50)
my_snn.fit(training_generator, eval_generator, epochs=100)